In [1]:
import open3d as o3d
import time, builtins, tempfile, datetime, os 
from BetaPose import utils, chemtools, representations; 

import pytraj as pt 
import numpy as np 
from scipy.stats import entropy 
from scipy.ndimage import gaussian_filter 
from scipy.spatial import distance_matrix 

from BetaPose import utils, cluster

FEATURIZER_PARMS = {
  # Mask of components 
  "MASK_INTEREST" : ":LIG,MDL", 
  "MASK_ENVIRONMENT" : ":1-221",
  
  # POCKET SETTINGS
  "VOXEL_DIMENSION" : [12, 12, 12],    # Unit: 1 (Number of lattice in one dimension)
  "CUBOID_LENGTH" : [8,8,8],           # Unit: Angstorm (Need scaling)
  
  # SEARCH SETTINGS
  "UPDATE_INTERVAL" : 1, 
  "CUTOFF": 18, 
}

# Example workflow

from BetaPose import trajloader, data_io
from BetaPose import features, featurizer


# Load multiple trajectories
trajs = "/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_001_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_002_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_003_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_004_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_005_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_006_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_007_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_008_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_009_traj.nc%/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_010_traj.nc%"
tops = ["/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C209CsDJQucZ_job_010_END.pdb"] * 10


trajs = "/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_001_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_002_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_003_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_004_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_005_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_006_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_007_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_008_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_009_traj.nc%/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_010_traj.nc%"
tops = ["/media/yzhang/MieT5/BetaPose_trajs/C209CsDJQucZ_job_008_END.pdb"] * 10
trajs = trajs.strip("%").split("%")
tloader = trajloader.TrajectoryLoader(trajs, tops);
trajectories = trajloader.TrajectoryLoader(trajs, tops); 

for traj in trajectories: 
  # Complete the trajectory information
  traj.strip(":T3P")
  print(traj.traj)
  #### traj.addcharge(); 
  
  # Initialize the featurizer since different trajectory might have distinct parameters
  # featurizer  = featurizer.Featurizer3D(FEATURIZER_PARMS); 
  featurizer  = featurizer.Featurizer3D(FEATURIZER_PARMS); 
  feature_mass = features.MassFeature(); 
  # NOTE: in this step, the feature hooks back the feature and could access the featurizer by feat.featurer
  
  featurizer.register_feature(feature_mass)   # i features
  featurizer.register_traj(traj)
  featurizer.register_frames(range(1,50,10))
  # featurizer.register_frames(range(1,50,10))  # j frames
  # featurizer.register_centers() # k centers 
  
  repr_traji, fpfh_traji, features_traji = featurizer.run_by_atom(traj.top.select("@CA&:50-70"), fbox_length=[6,6,6])
  
#   print(repr_traji.tolist())
  
  print(repr_traji.shape)
  print(fpfh_traji.shape)
  print(features_traji.shape)
#   featurizer.dump("repr_form", repr_traji, "/tmp/test.h5"); 
#   featurizer.dump("FPFH", np.array([(0,d) for d in fpfh_traji], dtype=object), "/tmp/test.h5"); 
  break
  
import pickle 
thedict = {
  "repr":repr_traji, 
  "fpfh":fpfh_traji, 
  "features":features_traji
}

with open('/tmp/test.pkl', 'wb') as f:
  # Write the object to the file
  pickle.dump(thedict, f)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loading the featurizer module
pytraj.Trajectory, 1001 frames: 
Size: 0.056316 (GB)
<Topology: 2517 atoms, 188 residues, 38 mols, PBC with box type = cubic>
           
Center [0 0 0]
Found the MSMS executable msms_i86_64Linux2_2.6.1/msms.x86_64Linux2.2.6.1
23-05-15T19:37:09   : [6 6 6]
23-05-15T19:37:09   : Frame 1: Generated 21 centers
23-05-15T19:37:15   : Centers 21 ; Feature vector:  (21, 1, 12, 12, 12)
23-05-15T19:37:15   : Frame 11: Generated 21 centers
23-05-15T19:37:21   : Centers 21 ; Feature vector:  (21, 1, 12, 12, 12)
23-05-15T19:37:21   : Frame 21: Generated 21 centers
23-05-15T19:37:28   : Centers 21 ; Feature vector:  (21, 1, 12, 12, 12)
23-05-15T19:37:28   : Frame 31: Generated 21 centers
23-05-15T19:37:34   : Centers 21 ; Feature vector:  (21, 1, 12, 12, 12)
23-05-15T19:37:34   : Frame 41: Generated 21 

# Viewpoint Feature histogram


In [ ]:
import numpy as np

def compute_vfh(points, normals, viewpoint, bins=45):
  # Compute the centroid
  centroid = np.mean(points, axis=0); 
  # Compute the direction from the centroid to each point
  directions = viewpoint - centroid; 
  # Compute the angles between the directions and the normals
  angles = np.arccos(np.sum(normals * directions, axis=1) / np.linalg.norm(directions, axis=1))
  # Compute the viewpoint component histogram
  vfh_hist_viewpoint, _ = np.histogram(angles, bins=bins, range=(0, np.pi))

  # Compute the Extended FPFH component for each point
  extended_fpfh = np.zeros((len(points), bins))

  for i, (point, normal) in enumerate(zip(points, normals)):
      k_neighbors = np.arange(len(points))
      k_neighbors = k_neighbors[k_neighbors != i]
      extended_fpfh[i] = compute_pfh(points, normals, i, k_neighbors, bins)

  # Compute the Extended FPFH histogram by summing up the histograms for all points
  vfh_hist_extended = np.sum(extended_fpfh, axis=0)

  # Normalize the histograms
  vfh_hist_viewpoint = vfh_hist_viewpoint / np.sum(vfh_hist_viewpoint)
  vfh_hist_extended = vfh_hist_extended / np.sum(vfh_hist_extended)

  # Concatenate the viewpoint and extended histograms to form the VFH
  vfh = np.concatenate((vfh_hist_viewpoint, vfh_hist_extended))

  return vfh


In [5]:
import numpy as np 
import open3d as o3d
import time
from BetaPose import utils

from matplotlib.cm import inferno

def calc_tm(roll, pitch, yaw, translate=[0, 0, 0]):
    # Precompute trigonometric functions
    cos_roll, sin_roll = np.cos(roll), np.sin(roll)
    cos_pitch, sin_pitch = np.cos(pitch), np.sin(pitch)
    cos_yaw, sin_yaw = np.cos(yaw), np.sin(yaw)

    # Generate rotation matrices
    Rx = np.array([[1, 0, 0], [0, cos_roll, -sin_roll], [0, sin_roll, cos_roll]])
    Ry = np.array([[cos_pitch, 0, sin_pitch], [0, 1, 0], [-sin_pitch, 0, cos_pitch]])
    Rz = np.array([[cos_yaw, -sin_yaw, 0], [sin_yaw, cos_yaw, 0], [0, 0, 1]])

    # Combine rotations
    R = Rx @ Ry @ Rz

    # Create the final transformation matrix
    H = np.eye(4); 
    H[:3, :3] = R; 
    H[:3, 3] = np.array(translate).ravel()

    return H


def transform_pcd(pcd, trans_mtx): 
  # Homogenize the point cloud (add a row of ones)
  homogeneous_pcd = np.hstack((pcd, np.ones((pcd.shape[0], 1))));
  # Apply the transformation matrix to the point cloud
  transformed_pcd = np.dot(homogeneous_pcd, trans_mtx.T);
  # Remove the homogeneous coordinate (last column)
  transformed_pcd = transformed_pcd[:, :3];
  return transformed_pcd
  

pcd = np.arange(102*3).reshape(-1,3)
# pcd = np.asarray([np.zeros(102),np.zeros(102), np.arange(102)]).T
print(pcd)
pcd_out = np.array(pcd)

t1 = time.perf_counter()

for r in np.linspace(0, np.pi*2, 20): 
  for p in np.linspace(0, np.pi*2, 20): 
    for z in np.linspace(0, np.pi*2, 20): 
      TransMatrix = utils.calc_tm(r, p , z)
      pcd2 = utils.transform_pcd(pcd, TransMatrix)
      pcd_out = np.vstack((pcd_out, pcd2))
  
  
print("Method1: ", time.perf_counter() - t1,)
# print(pcd2)


# import matplotlib.pyplot as plt 

point_cloud = o3d.geometry.PointCloud()

# Set the points of the point cloud object
point_cloud.points = o3d.utility.Vector3dVector(pcd_out)

thecmap = inferno(np.linspace(0,1,len(pcd_out)))[:, :3]
print(thecmap)

point_cloud.colors = o3d.utility.Vector3dVector(thecmap)


# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud])




[[  0   1   2]
 [  3   4   5]
 [  6   7   8]
 ...
 [297 298 299]
 [300 301 302]
 [303 304 305]]
Method1:  6.0915879950043745
[[1.46200e-03 4.66000e-04 1.38660e-02]
 [1.46200e-03 4.66000e-04 1.38660e-02]
 [1.46200e-03 4.66000e-04 1.38660e-02]
 ...
 [9.88362e-01 9.98364e-01 6.44924e-01]
 [9.88362e-01 9.98364e-01 6.44924e-01]
 [9.88362e-01 9.98364e-01 6.44924e-01]]


In [10]:
pcd = np.arange(102*3).reshape(-1,3)
np.vstack((pcd,pcd))

array([[  0,   1,   2],
       [  3,   4,   5],
       [  6,   7,   8],
       [  9,  10,  11],
       [ 12,  13,  14],
       [ 15,  16,  17],
       [ 18,  19,  20],
       [ 21,  22,  23],
       [ 24,  25,  26],
       [ 27,  28,  29],
       [ 30,  31,  32],
       [ 33,  34,  35],
       [ 36,  37,  38],
       [ 39,  40,  41],
       [ 42,  43,  44],
       [ 45,  46,  47],
       [ 48,  49,  50],
       [ 51,  52,  53],
       [ 54,  55,  56],
       [ 57,  58,  59],
       [ 60,  61,  62],
       [ 63,  64,  65],
       [ 66,  67,  68],
       [ 69,  70,  71],
       [ 72,  73,  74],
       [ 75,  76,  77],
       [ 78,  79,  80],
       [ 81,  82,  83],
       [ 84,  85,  86],
       [ 87,  88,  89],
       [ 90,  91,  92],
       [ 93,  94,  95],
       [ 96,  97,  98],
       [ 99, 100, 101],
       [102, 103, 104],
       [105, 106, 107],
       [108, 109, 110],
       [111, 112, 113],
       [114, 115, 116],
       [117, 118, 119],
       [120, 121, 122],
       [123, 124

In [7]:
_pcd = np.arange(100).reshape((-1,4))
c = np.mean(_pcd, axis = 0); 
print(c)

[48. 49. 50. 51.]


In [ ]:
import matplotlib.pyplot as plt 
import time


class VFH(object):
  """
  Parent class for PFH
  """

  def __init__(self, div, nneighbors, rad):
    """
    Pass in parameters
    """
    self._div = div
    self._nneighbors = nneighbors
    self._radius = rad

    self._error_list = []
    self._Rlist = []
    self._tlist = []
    self._pcd = np.array([])
    print("After initializaiton");

  def push_pcd(self, pcd, norm):
    if len(self._pcd) > 0:
      self._pcd = np.vstack(self._pcd, pcd);
      self._norm = np.vstack(self._norm, norm);
    else:
      self._pcd = np.array(pcd);
      self._norm = np.array(norm);
    self.N = len(self._pcd);
    self._kdtree = spatial.KDTree(self._pcd);

  def get_neighbors(self, pq):
    """
    Get k nearest neighbors of the query point pq from pc within the radius using KDTree.

    :param pq: Query point (1D numpy array of shape (3,))
    :param pc: Point cloud (2D numpy array of shape (N, 3))
    :return: Indices of the k nearest neighbors within the radius, distances of the k nearest neighbors
    """

    distances, indices = self._kdtree.query(pq, k=self._nneighbors+1, distance_upper_bound=self._radius)
    # Filter out neighbors with infinite distance (outside the search radius)
    valid_neighbors = np.isfinite(distances)
    indices = indices[valid_neighbors]
    distances = distances[valid_neighbors]

    return distances, indices

  def calc_fpfh(self):
    
    pass
  
  def calc_efpfh(self):
    pcd_center = np.mean(self._pcd, axis=0);

    for point_idx in range(self.N):
      pi = self._pcd[point_idx];
      ni = self._norm[point_idx];
      
      _, neighbor_idx = self.get_neighbors(pi);
      neighbors = neighbor_idx.tolist(); 
      neighbors.remove(point_idx);    # Remove the original query point
      # print(f"#### {len(neighbor_idx)} points queried")
      # print(point_idx, "=>" , neighbors)
      for nb_idx in neighbors: 
        u = self._norm[nb_idx]; 
        v = np.cross(pi-pcd_center, u)
        w = np.cross(u, v)
        dist = np.linalg.norm(vc)

        alpha = np.dot(v, u)
        phi = np.dot(u, vc)
        theta = np.arctan(np.dot(w, u) / np.dot(u, u))
        
  def calc_vpc(self, viewpoint, bins=128): 
    
    # Compute the relative position of the viewpoint to each point in the cloud
    rel_viewpoint = np.asarray(viewpoint) - self._pcd

    # Normalize the relative viewpoint vectors
    rel_viewpoint_normalized = rel_viewpoint / np.linalg.norm(rel_viewpoint, axis=1)[:, np.newaxis]

    # Calculate the angle between the normals and the relative viewpoint vectors
    cos_angles = np.sum(self._norm * rel_viewpoint_normalized, axis=1)
    angles = np.arccos(cos_angles)

    # Create the viewpoint component histogram
    hist, _ = np.histogram(angles, bins=bins, range=(0, np.pi))

    # Normalize the histogram
    hist_normalized = hist / np.sum(hist)
    plt.plot(np.arange(len(hist_normalized)), hist_normalized)
    plt.show()
    return hist_normalized

        


div = 2
nneighbors = 8
rad = 0.8

import open3d as o3d
import numpy as np
from scipy import spatial

mesh = o3d.io.read_triangle_mesh("/tmp/tmpz3t8q_hb_final.ply")

pcd = np.array(mesh.vertices)
# [1200:1800]
norms = np.array(mesh.vertex_normals)
# [1200:1800]
print(pcd.shape, norms.shape)

vfh_container = VFH(div, nneighbors, rad)
vfh_container.push_pcd(pcd, norms); 

# vfh_container.calc_viewpoint([10,10,10])
vfh_container.testvfh2([10,10,10])

# st = time.perf_counter()
# for i in range(1000): 
#   vfh_container.testvfh2([10,10,10])
# print(time.perf_counter() - st); 






In [3]:

import open3d as o3d
import matplotlib.pyplot as plt 
import open3d as o3d
import numpy as np
from scipy import spatial

In [36]:
np.arange(1,4)*np.arange(1,4)

np.arange(1,4).dot(np.arange(1,4))


14

In [1]:
import numpy as np
from BetaPose import utils

def uv(vector):
    return vector / np.linalg.norm(vector)

def ea_vectors(v1, v2):
    # Normalize both vectors
    v1_unit = v1/np.linalg.norm(v1)
    v2_unit = v2/np.linalg.norm(v2)

    # Calculate the cross product and dot product of the vectors
    cross = np.cross(v1_unit, v2_unit)
    dot = np.dot(v1_unit, v2_unit)

    # Calculate the angle between the vectors
    angle = np.arccos(np.clip(dot, -1.0, 1.0))

    # Calculate the axis of rotation
    axis = cross / np.linalg.norm(cross)

    # Calculate the quaternion
    qw = np.cos(angle / 2)
    qx, qy, qz = axis * np.sin(angle / 2)

    # Convert quaternion to Euler angles
    roll = np.arctan2(2 * (qw * qx + qy * qz), 1 - 2 * (qx * qx + qy * qy))
    pitch = np.arcsin(2 * (qw * qy - qz * qx))
    yaw = np.arctan2(2 * (qw * qz + qx * qy), 1 - 2 * (qy * qy + qz * qz))

    return roll, pitch, yaw

v1 = np.array([1, 0, 0])
v2 = np.array([3, 2, 1])

roll, pitch, yaw = ea_vectors(v1, v2)
print(f"Roll: {roll}, Pitch: {pitch}, Yaw: {yaw}")

Roll: -0.0823726211170212, Pitch: -0.27054976297857286, Yaw: 0.5880026035475675


In [70]:
tm = utils.calc_tm(roll, pitch, yaw); 
print(tm)
totrans = np.array([v2])
print(totrans)

ret = utils.transform_pcd(totrans, tm)
print(np.round(uv(ret),2), "diff", np.round((uv(ret) / uv(v1)), 2))

[[ 0.80178373 -0.53452248 -0.26726124  0.        ]
 [ 0.57111626  0.81703113  0.07928651  0.        ]
 [ 0.17598033 -0.21620787  0.96035675  0.        ]
 [ 0.          0.          0.          1.        ]]
[[3 2 1]]
[[0.29 0.92 0.28]] diff [[0.29  inf  inf]]


/tmp/ipykernel_10361/3694480922.py:7: RuntimeWarning: divide by zero encountered in true_divide
  print(np.round(uv(ret),2), "diff", np.round((uv(ret) / uv(v1)), 2))


In [39]:
import numpy as np

def compute_angles(v1, v2):
    # Normalize vectors
    v1_norm = v1 / np.linalg.norm(v1)
    v2_norm = v2 / np.linalg.norm(v2)

    # Compute the cross product of the two vectors
    cross = np.cross(v1_norm, v2_norm)

    # Compute the dot product of the two vectors
    dot = np.dot(v1_norm, v2_norm)

    # Compute the magnitude of the cross product and the angle between the two vectors
    mag_cross = np.linalg.norm(cross)
    angle = np.arccos(dot)

    # Compute the rotation matrix around the cross product vector
    if mag_cross != 0:
        cross_norm = cross / mag_cross
        cos_angle = np.cos(angle)
        sin_angle = np.sin(angle)
        R = np.array([[cos_angle + cross_norm[0]**2*(1-cos_angle), cross_norm[0]*cross_norm[1]*(1-cos_angle) - cross_norm[2]*sin_angle, cross_norm[0]*cross_norm[2]*(1-cos_angle) + cross_norm[1]*sin_angle],
                      [cross_norm[1]*cross_norm[0]*(1-cos_angle) + cross_norm[2]*sin_angle, cos_angle + cross_norm[1]**2*(1-cos_angle), cross_norm[1]*cross_norm[2]*(1-cos_angle) - cross_norm[0]*sin_angle],
                      [cross_norm[2]*cross_norm[0]*(1-cos_angle) - cross_norm[1]*sin_angle, cross_norm[2]*cross_norm[1]*(1-cos_angle) + cross_norm[0]*sin_angle, cos_angle + cross_norm[2]**2*(1-cos_angle)]])
    else:
        R = np.eye(3)

    # Compute the roll, pitch, and yaw angles from the rotation matrix
    yaw = np.arctan2(R[1, 0], R[0, 0])
    pitch = np.arctan2(-R[2, 0], np.sqrt(R[2, 1]**2 + R[2, 2]**2))
    roll = np.arctan2(R[2, 1], R[2, 2])

    return roll, pitch, yaw
  
eas = compute_angles(v1,v2)
tm = utils.calc_tm(*eas); 
ret = utils.transform_pcd(np.array([v2]), tm)
print(ret.round(2))



[[-0.82  0.96  0.64]]
